# Natual language approach to assess takeout habits

### Setup

In [1]:
# Import packages
import os
import json
import datetime
from datetime import timedelta, datetime
import pandas as pd
from dotenv import load_dotenv

from agno.agent import Agent
from agno.models.groq import Groq
from agno.tools import Toolkit

In [2]:
load_dotenv('../.env')
api_key = os.getenv('GROQ_API_KEY')

We will configure multiple LLMs using [Agno Agents](https://github.com/agno-agi/agno), each requiring a Groq API Key for access which you can create [here](https://console.groq.com/keys). These models include different versions of Meta's LLaMA 3 and other specialized models like Google's Gemma 2 and Mixtral. Each model will be used by different agents to generate diverse outputs for the MLB game recap.

In [3]:
llm_model = Groq(id="llama-3.3-70b-versatile", api_key=api_key)

In [4]:
import pandas as pd
grubhub = pd.read_csv('grubhub.csv', 
                      names=['na','restaurant','price','date'])
grubhub.drop('na',axis=1,inplace=True)
grubhub['date'] = pd.to_datetime(grubhub['date'])
grubhub

,restaurant,price,date
0,Little Caesars Pizza,21.24,2025-01-18
1,Five Guys,32.93,2025-01-05
2,Maiwand Kabob,53.43,2024-12-29
3,Chipotle,51.06,2024-12-27
4,Five Guys,32.93,2024-12-21
...,...,...,...
56,Maiwand Kabob,60.63,2024-02-06
57,An Loi,47.42,2024-02-01
58,Soul Boat,34.68,2024-01-22
59,Bonchon,25.83,2024-01-12


### Define Tools

In [5]:
def helper(restaurant_name: str, start_date: str, end_date: str) -> pd.DataFrame:
    name_match = True if not restaurant_name else grubhub['restaurant'] == restaurant_name
    date_match = (grubhub['date'] >= start_date) & (grubhub['date'] <= end_date)
    matches = grubhub[name_match & date_match]

    return matches

In [6]:
def get_frequency(restaurant_name: str = '', start_date: str = '2000-01-01', end_date: str = '2026-01-01') -> str:
    """Gets the number of times a restaurant appears in the data
    on or inbetween the given dates
    """
    matches = helper(restaurant_name, start_date, end_date)
    retval= {
        'frequency': matches.shape[0]
    }
    return json.dumps(retval)

print(get_frequency(start_date='2024-02-01', end_date='2024-02-28', restaurant_name='An Loi'))
print(get_frequency(start_date='2024-02-01', end_date='2024-02-28'))


{"frequency": 2}
{"frequency": 6}


In [7]:
def get_cost(restaurant_name: str = '', start_date: str = '2000-01-01', end_date: str = '2026-01-01') -> str:
    """Get the total cost of meals from a restaurant within a date range
    """
    matches = helper(restaurant_name, start_date, end_date)
    
    retval = {
        'total_cost': matches['price'].sum()
    }
    return json.dumps(retval)

print(get_cost(start_date='2024-02-01', end_date='2024-02-28', restaurant_name='An Loi'))


{"total_cost": 74.07}


### Define Agents

In [8]:
takeout_q = Agent(
    model=llm_model,
    description="An agent that answers questions about takeout history",
    instructions = [
        f"Parse the restaurant name (use '' if user does not specify), start date, and end date from the user question.",
        "Use the tools to answer questions about my takeout habits",    
    ],
    tools=[get_frequency, get_cost],
    show_tool_calls=True
)

In [9]:
user_prompt = {
    0: 'how many times did I eat at An Loi during the month of August 2024?',
    1: 'how many times did I order takeout during the month of August 2024?',
    2: 'how much did I spend at An Loi during the month of June 2024?',
    3: 'how much did I spend on takeout in July of 2024?',
    4: 'did I spend more in takeout in October or November of 2024?',
    5: 'how much did I spend on takeout in January 2024 and February 2024?',
    6: 'how much did I spend on takeout in January 2024?'

}

In [10]:
%%time
result = takeout_q.run(user_prompt[5], stream=False)
print(result.content)

You spent a total of $328.90 on takeout in January and February 2024.
CPU times: user 81.9 ms, sys: 22.8 ms, total: 105 ms
Wall time: 1.28 s


In [11]:
get_cost(start_date='2024-01-01', end_date='2024-01-31')
get_cost(start_date='2024-01-01', end_date='2024-02-28')

'{"total_cost": 328.90000000000003}'

## Try the same thing with Toolkit interface

In [12]:
class TakeoutTools(Toolkit):
    def __init__(self):
        super().__init__(name="takeout_tools")
        setattr(self.__class__, 'get_frequency', classmethod(get_frequency))
        setattr(self.__class__, 'get_cost', classmethod(get_cost))
        self.register(self.get_frequency)
        self.register(self.get_cost)
    

In [13]:
toolkit_takeout_q = Agent(
    model=llm_model,
    description="An agent that answers questions about takeout history",
    instructions = [
        f"Parse the restaurant name (use '' if user does not specify), start date as yyyy-MM-dd, and end date as yyyy-MM-dd from the user question.",
        "Use the available tools to answer questions about takeout history",    
    ],
    tools=[TakeoutTools()],
    show_tool_calls=True,
    markdown=False
)

In [14]:
%%time
user_prompt = 'how many times did I eat at An Loi during the month of February 2024?'
user_prompt = 'how many times did I eat out during the month of February 2024?'
result = toolkit_takeout_q.run(user_prompt, stream=False)
print(result.content)

Exception ignored in: <function SyncHttpxClientWrapper.__del__ at 0x11ca28d60>
Traceback (most recent call last):
  File "/Users/robertholder/Documents/installs/miniconda3/envs/agnoenv/lib/python3.11/site-packages/groq/_base_client.py", line 770, in __del__
    if self.is_closed:
       ^^^^^^^^^^^^^^
  File "/Users/robertholder/Documents/installs/miniconda3/envs/agnoenv/lib/python3.11/site-packages/httpx/_client.py", line 228, in is_closed
    return self._state == ClientState.CLOSED
           ^^^^^^^^^^^
AttributeError: 'SyncHttpxClientWrapper' object has no attribute '_state'
Exception ignored in: <function SyncHttpxClientWrapper.__del__ at 0x11ca28d60>
Traceback (most recent call last):
  File "/Users/robertholder/Documents/installs/miniconda3/envs/agnoenv/lib/python3.11/site-packages/groq/_base_client.py", line 770, in __del__
    if self.is_closed:
       ^^^^^^^^^^^^^^
  File "/Users/robertholder/Documents/installs/miniconda3/envs/agnoenv/lib/python3.11/site-packages/httpx/_cli

You ate out 6 times during the month of February 2024.
CPU times: user 61.2 ms, sys: 7.99 ms, total: 69.2 ms
Wall time: 1.07 s


In [15]:
result


RunResponse(content='You ate out 6 times during the month of February 2024.', content_type='str', thinking=None, reasoning_content=None, event='RunResponse', messages=[Message(role='system', content="An agent that answers questions about takeout history\n<instructions>\n- Parse the restaurant name (use '' if user does not specify), start date as yyyy-MM-dd, and end date as yyyy-MM-dd from the user question.\n- Use the available tools to answer questions about takeout history\n</instructions>", name=None, tool_call_id=None, tool_calls=None, audio=None, images=None, videos=None, files=None, audio_output=None, image_output=None, thinking=None, redacted_thinking=None, provider_data=None, citations=None, reasoning_content=None, tool_name=None, tool_args=None, tool_call_error=None, stop_after_tool_call=False, add_to_agent_memory=True, from_history=False, metrics=MessageMetrics(input_tokens=0, output_tokens=0, total_tokens=0, audio_tokens=0, input_audio_tokens=0, output_audio_tokens=0, cached

In [16]:
get_frequency(start_date='2024-02-01', end_date='2024-02-29')

'{"frequency": 6}'